# Semi-Automated Protected Attributes Detection over Sample/Label Biases

<br>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import itertools
import pandas as pd
import numpy as np
import fairlens as fl

import statsmodels.api as sm

from sklearn.datasets import fetch_openml
from scipy.stats import entropy, pearsonr, chi2_contingency, pointbiserialr, f_oneway, variation, kruskal
from statsmodels.stats.oneway import effectsize_oneway
from statsmodels.formula.api import ols
from scipy.stats.contingency import association
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

from packages.fairness_report import FairnessReport

<br>

## Download datasets

In [ ]:
"""
DATASETS = {
    45068: '43141_adult',
    46356: '46356_german_credit',
    45069: '45069_diabetes',
    43904: '43904_law_bar'   
}

for k in DATASETS:
    dataset = fetch_openml(data_id=k, as_frame=True)
    dataset.data.to_csv(DATASETS[k] + '_raw.csv', index=False)
"""

<br>

## Results reproduction 

In [ ]:
DATASETS = [
    45068,   # adult                         JOB/INCOME
    44096,   # GermanCreditData              FINANCE/BANKING
    45069,   # Diabetes130US                 HEALTH
    43904    # law-school-admission-bianry   EDUCATION
]


class Dsettings:
    def __init__(self, target, privileged, target_description, favorable_label, unfavorable_label):
        self.target = target
        self.privileged = privileged
        self.target_description = target_description
        self.favorable_label = favorable_label
        self.unfavorable_label = unfavorable_label

DSETTINGS = {
    0: Dsettings('class', '>50K', 'People earning over 50.000$', 1, 0),
    1: Dsettings('class', 1, 'People who were classified as good customers for a loan', 1, 0),
    2: Dsettings('class', 'NO', 'People NOT readmissioned to the hospital', 0, 1),
    3: Dsettings('ugpagt3', 'TRUE', 'People with admission (ugpa > 3)', 1, 0)
}

DRANKINGS = {}

<br>

### Repeat for each dataset

In [ ]:
dataset_id = 0
bins = 6

dataset = fetch_openml(data_id=DATASETS[dataset_id], as_frame=True)
df = dataset.data
    
df = df.replace('nan', np.nan)
df = df.dropna()    

if dataset_id == 2:
    dataset.target.loc[dataset.target != 'NO'] = 'YES'
        
df_copy = df.copy()
for c in df_copy.columns:
    df_copy[c] = df_copy[c].astype(str)
    
protected_attributes = list(fl.sensitive.detect_names_df(df_copy, deep_search=True).keys())

for pa in protected_attributes:
    if pa.lower() == 'age':
        try:
            df = df[df[pa] <= 100]
        except:
            # already categorised
            continue
        
    if df[pa].dtype == 'category':
        df[pa] = df[pa].astype('object')
        
    if df[pa].dtype != 'object':        
        if len(pd.unique(df[pa])) > 10:
            df[pa] = pd.cut(df[pa], bins=bins)
        df[pa] = df[pa].astype('object')
        
for c in df.columns:
    if (df[c].dtype == 'category'):
        df[c] = df[c].astype('object')
    elif (len(pd.unique(df[c])) <= 5):
        df[c] = df[c].astype('object') 
        
    if (df[c].dtype == 'int64' or df[c].dtype == 'float64') and (len(pd.unique(df[c])) > 20):
        df[c] = pd.cut(df[c], bins=bins)
        df[c] = df[c].astype('object')
        
df.head()

<br>

#### Normalization

In [ ]:
# normalize the data
numeric_data = df.select_dtypes(include=['int64', 'float64', 'uint8'])
categorical_data = df.select_dtypes(include=['object', 'category'])

try:
    categorical_data[dataset.target_names[0]] = dataset.target
except:
    # that means that target attribute is the last column in a dataframe
    pass
    
no_numeric = False
try:
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(numeric_data)
except:
    # no numerical data
    no_numeric = True
    pass

encoder = LabelEncoder()

if no_numeric:
    data = categorical_data.apply(encoder.fit_transform)
    data = data.astype(object)
    categorical_present = True
else:
    try:
        encoded_data = categorical_data.apply(encoder.fit_transform)
        encoded_data[encoded_data.columns.tolist()] = encoded_data[encoded_data.columns.tolist()].astype(np.object)
        data = pd.concat([pd.DataFrame(scaled_data, columns=numeric_data.columns), encoded_data.reset_index(drop=True)], axis=1)
        categorical_present = True
    except:
        # no categorical attritbutes present in the observed dataset -> data = scaled numeric data
        categorical_present = False
        data = pd.DataFrame(scaled_data, columns=numeric_data.columns)

data = data.drop_duplicates()
data.head()

In [ ]:
if categorical_present:
    categorical_data = df.select_dtypes(include=['object', 'category'])
    
    try:
        categorical_data[dataset.target_names[0]] = dataset.target
    except:
        # last column is the target label/attribute
        pass

    for c in categorical_data.columns:
        le = LabelEncoder()
        le.fit(categorical_data[c])
        le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

        #print()
        #print('Attribute: ' + c)
        #print(le_name_mapping)
#else:
    #print('No categorical attributes found in the dataset!')

<br>

#### (Hidden) Correlation

In [ ]:
significance_matrix = {}
correlation_matrix = pd.DataFrame(index=data.columns, columns=data.columns)
for col1 in data.columns:
    for col2 in data.columns:
        if col1 != col2:
            if (data[col1].dtype == np.object and data[col2].dtype == np.object):
                # Chi2 test + Cramer's V test
                contingency_table = pd.crosstab(data[col1], data[col2])
                chi2, p, dof, expected = chi2_contingency(contingency_table)
                
                if (p < 0.05):
                    cramers_v_value = association(contingency_table)
                    correlation_matrix.at[col1, col2] = cramers_v_value
                    significance_matrix[(col1, col2)] = p
            elif ((data[col1].dtype == np.object and data[col2].dtype != np.object) or
                  (data[col1].dtype != np.object and data[col2].dtype == np.object)):
                if (data[col1].dtype == np.object):
                    cat = data[col1]
                    numeric = data[col2]
                else:
                    cat = data[col2]
                    numeric = data[col1]
                    
                # One-Way ANOVA with effect size (multi-class categorical variable) -> omega squared effect size
                if (len(pd.unique(cat)) > 2):
                    q = data[[numeric.name, cat.name]]
                    q[cat.name] = pd.to_numeric(q[cat.name])
                    
                    model = ols('Q(\"' + numeric.name + '\")' + ' ~ C(Q(\"' + cat.name + '\"))', data=q).fit()
                    aov = sm.stats.anova_lm(model, typ=2)
                    
                    if (aov['PR(>F)'].iloc[0] < 0.05):
                        aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
                        aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
                        aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
                        correlation_matrix.at[col1, col2] = aov['omega_sq'].iloc[0]
                        significance_matrix[(col1, col2)] = aov['PR(>F)'].iloc[0]
                else:
                    # Point-Biserial (binary categorical variable)
                    pointbiserial_corr, pointbiserial_p_value = pointbiserialr(cat, numeric)
                    if (pointbiserial_p_value < 0.05):
                        correlation_matrix.at[col1, col2] = pointbiserial_corr
                        significance_matrix[(col1, col2)] = pointbiserial_p_value
            else:                
                # Pearson's correlation
                coef, p_value = pearsonr(data[col1], data[col2])
                if (p_value < 0.05):
                    correlation_matrix.at[col1, col2] = coef
                    significance_matrix[(col1, col2)] = p_value

In [ ]:
below_main_diagonal = correlation_matrix.where(np.tril(np.ones(correlation_matrix.shape), k=-1).astype(np.bool))
q = below_main_diagonal.fillna(0).stack()

top_largest = q.nlargest(15)
top_smallest = q.nsmallest(5)

candidates = pd.concat([top_largest, top_smallest])

candidates = pd.DataFrame(candidates, columns=['corr'])
candidates['abs_corr'] = abs(candidates['corr']) 
candidates = candidates.sort_values(by='abs_corr', ascending=False)
candidates = candidates['corr'].head(15)


try:
    exclude = list(protected_attributes) + [dataset.target_names[0]]
except:
    exclude = list(protected_attributes) + [data.columns[-1]]

attribute_pairs = candidates.index.values.tolist()

result = pd.Series()
consumed_attributes = []
for i, attribute_pair in enumerate(attribute_pairs):
    att1 = attribute_pair[0]
    att2 = attribute_pair[1]

    if (att1 not in protected_attributes and att2 not in protected_attributes):
        continue
    if (att1 in exclude or att1 in consumed_attributes) and (att2 in exclude or att2 in consumed_attributes):
        continue    
    if (att1 not in exclude and att1 not in consumed_attributes):
        consumed_attributes.append(att1)
    if (att2 not in exclude and att2 not in consumed_attributes):
        consumed_attributes.append(att2)

    result = pd.concat([result, pd.Series(data={candidates.index[i]: candidates[i]})])
    
result

In [ ]:
q = pd.unique(list(itertools.chain.from_iterable(result.index.values.tolist())))
focus_attributes = [a for a in q if a not in exclude]
focus_attributes

<br>

#### Metrics report

In [ ]:
fr = FairnessReport(dataset, df, data)

focus = protected_attributes + focus_attributes
stats = fr.detect_protected_attributes(
    data[focus], 
    DSETTINGS[dataset_id].favorable_label, 
    DSETTINGS[dataset_id].unfavorable_label)
stats

<br>

#### Metrics ranking

In [ ]:
# "higher is better" in the context how sensitive an attribute actually is 
# ranking score   -> the least sensitive attribute gets the best score (e.g., 1) for observed metric
# ranking overall -> the most sensitive attribute will have the highest sum of ranking scores, while
#                    the least sensitive the lowest sum of ranking scores
higher_is_better = {
    'entropy': True,                           # the higher the entropy, the less sensitive an attribute is
    'imbalance_ratio': False,                  # the higher the imbalance ratio, the more sensitive an attribute is
    'imbalance_degree': False,                 # the higher the imbalance degree, the more sensitive an attribute is
    'statistical_parity_difference': False,    # the higher the (absolute) statistical parity difference, the more sensitive an attribute is
    'disparate_impact_ratio': False,           # the higher the difference ("DIR - 1" observation), the more sensitive an attribute is (difference of 0 would indicate perfect fairness)
    'smoothed_edf': True                       # the higher the smoothed EDF, the less sensitive an attribute (value of 1 would indicate perfect fairness)
}

def calculate_rank(values):
    sorted_values = values.sort_values(ascending=not higher_is_better[metric])
    ranks = sorted_values.rank(method='dense', ascending=not higher_is_better[metric])
    return ranks

ranks = stats[stats.columns[3:]]

drop = []
for metric in ranks.columns:
    try:
        if metric == 'disparate_impact_ratio':
            ranks[metric] = calculate_rank(abs(ranks[metric] - 1))
        elif metric == 'imbalance_degree':
            ranks[metric] = calculate_rank(ranks[metric] / stats['total_classes'])
        elif metric == 'statistical_parity_difference':
            ranks[metric] = calculate_rank(abs(ranks[metric]))
        else:
            ranks[metric] = calculate_rank(ranks[metric])
    except:
        drop.append(metric)

if len(drop) > 0:
    ranks = ranks.drop(drop, axis=1)
    
ranks = ranks.astype(int)
ranks

In [ ]:
distribution_metrics = ['entropy', 'imbalance_ratio', 'imbalance_degree']
aif360_metrics = ['statistical_parity_difference', 'disparate_impact_ratio', 'smoothed_edf']

dist_rankings = pd.DataFrame(ranks[distribution_metrics].sum(axis=1).sort_values(ascending=False), columns=['RS*'])
dist_rankings['R*'] = list(range(1, len(dist_rankings) + 1))

aif360_rankings = pd.DataFrame(ranks[aif360_metrics].sum(axis=1).sort_values(ascending=False), columns=['RS**'])
aif360_rankings['R**'] = list(range(1, len(aif360_rankings) + 1))

rankings_all = pd.DataFrame(ranks.sum(axis=1).sort_values(ascending=False), columns=['RS'])
rankings_all['R'] = list(range(1, len(rankings_all) + 1))

dataset_rankings = dist_rankings.join(aif360_rankings)
dataset_rankings = dataset_rankings.join(rankings_all)
dataset_rankings = dataset_rankings.sort_values(by='R')

print('Legend:')
print('------------------------------')
print('RS* = Total Ranking Score for ranking scores which include only \'distribution\' metrics')
print('      (e.g., entropy, IR & ID)')
print('R* = Ranking based on total ranking scores coming only from \'distribution\' metrics')
print('      (e.g., entropy, IR & ID)')
print()
print('RS** = Total Ranking Score for ranking scores which include only \'AIF360\' metrics')
print('      (e.g., SPD, DIR & SEDF)')
print('R** = Ranking based on total ranking scores coming only from \'AIF360\' metrics')
print('      (e.g., SPD, DIR & SEDF)')
print()
print('RS = Total Ranking Score for ALL ranking scores')
print('R = Ranking based on total ranking scores coming from ALL metrics')

dataset_rankings

In [ ]:
DRANKINGS[dataset_id] = stats.join(dataset_rankings).sort_values(by='R')

<br>

## Results Overview

In [ ]:
for e in DRANKINGS:
    print("DID: " + str(DATASETS[e]))
    display(DRANKINGS[e])
    DRANKINGS[e].to_csv(str(DATASETS[e]) + '_results.csv', sep='\t')
    print()

<br>

## Verification Method

In [ ]:
bld = BinaryLabelDataset(
    df=data,
    label_names=dataset.target_names,
    protected_attribute_names=focus,
    favorable_label=DSETTINGS[dataset_id].favorable_label, 
    unfavorable_label=DSETTINGS[dataset_id].unfavorable_label
)

In [ ]:
dataset_original_train, dataset_original_test = bld.split([0.7], shuffle=True)

In [ ]:
model = RandomForestClassifier()
model.fit(dataset_original_train.features, dataset_original_train.labels.ravel())

In [ ]:
y_pred = model.predict(dataset_original_test.features)

classified_dataset = dataset_original_test.copy()
classified_dataset.labels = y_pred

for attribute in focus:
    majority_classes = fr.get_majority_classes(data[attribute])
    minority_classes = fr.get_minority_classes(data[attribute])
    
    privileged_groups = [{attribute: v} for v in majority_classes]
    unprivileged_groups = [{attribute: v} for v in minority_classes]
    
    metric = ClassificationMetric(
        dataset_original_test, 
        classified_dataset, 
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups)

    print("-----------------------------------------------------")
    print("Attribute in focus: " + attribute)
    print("False negative rate difference: " + str(round(metric.false_negative_rate_difference(), 2)))
    print()